# Imviz Demonstration Notebook
## Roman/ASDF Edition

This notebook demonstrates the Imviz API in the Notebook setting when using simulated observations from The Nancy Grace Roman Space Telescope. UI equivalents for these actions, as well as additional documentation about Imviz, can be found here: https://jdaviz.readthedocs.io/en/latest/imviz/

Import modules needed for this notebook.

In [ ]:
import os
import warnings
import tempfile
import asdf

import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.table import Table
from astroquery.mast import Observations
from photutils import CircularAperture, SkyCircularAperture
from regions import PixCoord, CirclePixelRegion, CircleSkyRegion

from jdaviz import Imviz

We also need this to display Matplotlib in the notebook later.

In [ ]:
%matplotlib inline

We create an Imviz instance and grab the default viewer instance as well.

In [ ]:
imviz = Imviz()
# default image viewer is named 'imviz-0'

Now we download some data, using `astroquery` to retrieve the files from
[MAST](https://masttest.stsci.edu/). By default the downloaded files go to your
temp directory, and thus may eventually be garbage collected (deleted) by your system.
If you would like to have the file permanently, simply uncomment the second line below
and provide the directory path on your system where you would like the file stored.

One other thing to note about retrieving MAST data through astroquery is that it caches
the data by default. It is possible for files to be updated in MAST with more recent calibrations
but remain the same size, in which case your local cached file would not be replaced by the new
version. You can turn off caching by setting `cache=False` in the `download_file` call to
force it to re-download the file if desired.

In this example, we use two of the JWST NIRCam
observations of the Cartwheel Galaxy. These are large files (~500 MB each), so they make take
some time to download, but after the first time running this notebook they will be cached locally.
If you want to look at images from other filters, you can uncomment the lines with those filenames.
If you want an example using smaller files, see the ImvizDitheredExample notebook, which
includes the same workflow as this one, but using two HST ACS files that are much smaller
than the JWST files used here.

After downloading, each file is loaded into Imviz. We ignore some warnings that appear during parsing.

In [ ]:
data_dir = tempfile.gettempdir()

urls = {
    'WFI01_cal.asdf': 'https://stsci.box.com/shared/static/jaf6oj7j3kwr7wh7kqfxmj81a31pcjtv.asdf',
    #'WFI02_cal.asdf': 'https://stsci.box.com/shared/static/ife8tbvh8xb04xxun17y3krdzz2t13o7.asdf'
}

extension = 'data'

for viewer_name, (fn, url) in zip(imviz.app.get_viewer_reference_names(), urls.items()):
    path = os.path.join(data_dir, fn)
    
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        result = Observations._download_file(url, local_filepath=path, cache=True)

        # we will load only one extension ("data") from an NDArray
        path_and_extension = f'{path}[{extension}]'

        data_label = fn.split('.')[0]

        # load the observations into the helper
        imviz.load_data(
            path_and_extension, data_label=data_label, 
            show_in_viewer=False
        )

    # now add the nth "data" image to the nth viewer:
    imviz.app.add_data_to_viewer(viewer_name, imviz.app.data_collection[-1].label)

Then, we visualize the data and start off by looking at some of the basic features:

In [ ]:
imviz.show()

Imviz will initially display Roman images zoomed to fit the display. However, since each detector's images have 4k resolution, the default Imviz zoom makes it hard to tell if the colormap stretch is appropriate.

Panning and zooming is possible by showing the viewer toolbar and clicking on the '+'-shaped icon, then dragging around in the image and using scrolling to zoom in and out. To change the stretch and colormap, show the **Layer** options accessible through the last icon in the viewer toolbar.

We can also change these programmatically, for example the stretch:

In [ ]:
for viewer_name in imviz.app.get_viewer_reference_names():
    viewer = imviz.app.get_viewer(viewer_name)

    # set colormap and stretch:
    viewer.stretch = 'sqrt'
    viewer.set_colormap('Viridis')
    viewer.cuts = (0.8, 3)
    
    # set zoom level
    viewer.zoom(5)

Note also that in the above example there are mouse-over coordinates visible by default.

It possible to make selections/regions in images and export these to Astropy regions. Click on the viewer toolbar then click on the circular selection tool, and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = imviz.get_interactive_regions()

In [ ]:
regions

Since the region is an Astropy region, we can e.g. convert it to a mask:

In [ ]:
mask = regions['Subset 1'].to_mask(mode='subpixels')

In [ ]:
data = imviz.app.get_data_from_viewer('imviz-0', imviz.app.data_collection[0].label)

In [ ]:
plt.imshow(mask.to_image(data.data.shape), origin='lower')

It is also possible to programmatically pass a `regions` shape, a `photutils` aperture shape, or a Numpy mask into Imviz.

In [ ]:
c = SkyCoord('05h37m44.33s -69d08m30.484s')

# photutils aperture
my_aper = CircularAperture((2402, 2625), r=20)
my_aper_sky = SkyCircularAperture(c, 2 * u.arcsec)

# regions shape
my_reg = CirclePixelRegion(center=PixCoord(x=1526, y=1063), radius=40)
my_reg_sky = CircleSkyRegion(c, Angle(2, u.arcsec))

# Numpy mask
idx = (np.array([350, 350, 350, 350, 350, 350, 351, 351, 351, 351, 352, 352, 352,
                 352, 352, 352, 352, 352, 352, 352, 353, 353, 353, 353, 353, 353,
                 353, 353, 353, 353, 353, 353, 354, 354, 354, 354, 354, 354, 354,
                 354, 355, 355, 355, 355, 355, 355, 355, 355, 356, 356, 356, 356,
                 356, 356, 356, 357, 357, 358, 358]),
       np.array([353, 354, 355, 356, 357, 358, 350, 352, 359, 361, 350, 352, 353,
                 354, 355, 356, 357, 358, 359, 361, 350, 351, 352, 353, 354, 355,
                 356, 357, 358, 359, 360, 361, 351, 352, 354, 355, 356, 357, 359,
                 360, 352, 353, 354, 355, 356, 357, 358, 359, 352, 353, 354, 355,
                 356, 357, 358, 353, 358, 352, 359]))
my_mask = np.zeros(data.data.shape, dtype=np.bool_)
my_mask[idx] = True

my_regions = [my_aper, my_aper_sky, my_reg, my_reg_sky, my_mask]
imviz.load_regions(my_regions)

You can also programmatically control the viewer. Note that the following will fail
if you have blinked between the two images and have data `B` active, due to the
smaller size of that image.

In [ ]:
# Center the image on given pixel position.
viewer.center_on((2402, 2625))  # X, Y (0-indexed)

In [ ]:
# Move the image with the given pixel offsets.
viewer.offset_by(500, -100)  # dX, dY

We can convert the coordinates of the interactive subset from pixel to world coordinates like so:

In [ ]:
center = regions['Subset 1'].center
data.wcs.pixel_to_world([center.x], [center.y]).to_string('hmsdms')

In [ ]:
# Center the image on given sky coordinates.
sky = SkyCoord(ra=84.43470962, dec=-69.14180132, unit=(u.deg, u.deg))
viewer.center_on(sky)

In [ ]:
# Move the image with the given sky offsets.
viewer.offset_by(0.5 * u.arcsec, -1.5 * u.arcsec)

You can programmatically zoom in and out.

Zoom level:

* 1 means real-pixel-size.
* 2 means zoomed in by a factor of 2.
* 0.5 means zoomed out by a factor of 2.
* 'fit' means zoomed to fit the whole image width into display.

In [ ]:
# Get the current zoom level.
viewer.zoom_level

In [ ]:
# Set the zoom level directly.
viewer.zoom_level = 1

In [ ]:
# Set the relative zoom based on current zoom level.
viewer.zoom(2)

It is also possible to programmatically add non-interactive markers to the image.

In [ ]:
# Add 20 randomly generated X,Y (0-indexed) w.r.t. reference image
# using default marker properties.
t_xy = Table({'x': np.random.randint(0, 4000, 20),
              'y': np.random.randint(0, 4000, 20)})
viewer.add_markers(t_xy)

You could customize marker color, alpha, size, and fill with values that Glue supports.

In [ ]:
viewer.marker = {'color': 'green', 'alpha': 0.8, 'markersize': 10, 'fill': False}

In [ ]:
np.random.seed(42)
# Mark some sky coordinates using updated marker properties.
t_sky = Table({'coord': [c.spherical_offsets_by(*np.random.normal(scale=0.1, size=2) * u.arcmin)
                         for i in range(5)]})
viewer.add_markers(t_sky, use_skycoord=True, marker_name='my_sky')
t_sky

When you do not need the markers anymore, you could remove them by name.

In [ ]:
viewer.remove_markers(marker_name='my_sky')

You can also remove all the markers.

In [ ]:
viewer.reset_markers()

You can save the active display as PNG file.

In [ ]:
viewer.save('myimage.png')

If you want a second viewer, run the following cell *or* click on the "picture with +" icon on top of the image viewer.

If you click the icon, the viewer name will be auto-generated. Instead of running the next cell, you can run this instead:

    viewer_2_name = 'imviz-1'
    viewer_2 = imviz.app.get_viewer_by_id(viewer_2_name)

In [ ]:
viewer_2_name = 'my-viewer-1'

# Do not pass in viewer_name if you want it auto-generated.
# If auto-generated, viewer name will be 'imviz-1'.
viewer_2 = imviz.create_image_viewer(viewer_name=viewer_2_name)

You can also programmatically control the new viewer, as follows.

In [ ]:
# Show the first image in the second viewer.
imviz.app.add_data_to_viewer(viewer_2_name, "WFI01_cal[data]")

In [ ]:
viewer_2.zoom(4)

In [ ]:
viewer_2.cuts = '95%'

In [ ]:
viewer_2.center_on((995, 1000))

To destroy this new viewer, run the following cell *or* click the "x" that is right next to the viewer ID.

In [ ]:
imviz.destroy_viewer(viewer_2_name)